# Ứng dụng AI đánh giá phản hồi học phần từ sinh viên theo cảm xúc và chủ đề

Đề tài này nhằm xây dựng một hệ thống sử dụng AI để phân tích phản hồi học phần của sinh viên, xác định cảm xúc (tích cực, tiêu cực, trung lập) và phân loại chủ đề phản hồi (giảng viên, nội dung học phần, cơ sở vật chất, v.v.). Dữ liệu sử dụng là bộ `uitnlp/vietnamese_students_feedback` từ Hugging Face.

## Mục tiêu
- Phân loại cảm xúc của phản hồi.
- Phân loại chủ đề phản hồi.
- Đánh giá hiệu quả mô hình và demo dự đoán.


In [ ]:
import datasets

# Initialize and prepare the dataset
vsf = VietnameseStudentsFeedback()
vsf.download_and_prepare()

# Load the dataset
ds = vsf.as_dataset()

# Convert to pandas dataframes for easier handling
import pandas as pd
train_df = pd.DataFrame(ds["train"])
validation_df = pd.DataFrame(ds["validation"])
test_df = pd.DataFrame(ds["test"])

print("Dataset structure:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(validation_df)}")
print(f"Test samples: {len(test_df)}")
print("\nSample data:")
print(train_df.head())

/Users/ducqhle/Library/Python/3.13/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Dataset scripts are no longer supported, but found vietnamese_students_feedback.py

In [ ]:
import re
import pandas as pd

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[\n\r]", " ", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Áp dụng tiền xử lý
dataset_cleaned = dataset.map(lambda x: {"text": preprocess_text(x["text"])})


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# Tải tokenizer và model PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model_emotion = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3)

# Tokenize dữ liệu
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_emotion = dataset_cleaned.map(tokenize_function, batched=True)

# Thiết lập huấn luyện
training_args = TrainingArguments(
    output_dir="./results_emotion",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs_emotion",
    logging_steps=10,
)

trainer_emotion = Trainer(
    model=model_emotion,
    args=training_args,
    train_dataset=tokenized_emotion["train"],
    eval_dataset=tokenized_emotion["test"],
)

# Huấn luyện mô hình
trainer_emotion.train()


In [ ]:
# Tạo mô hình phân loại chủ đề (giả sử có 5 chủ đề)
model_topic = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=5)

# Tokenize dữ liệu
tokenized_topic = dataset_cleaned.map(tokenize_function, batched=True)

# Thiết lập huấn luyện
training_args_topic = TrainingArguments(
    output_dir="./results_topic",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs_topic",
    logging_steps=10,
)

trainer_topic = Trainer(
    model=model_topic,
    args=training_args_topic,
    train_dataset=tokenized_topic["train"],
    eval_dataset=tokenized_topic["test"],
)

# Huấn luyện mô hình
trainer_topic.train()


In [ ]:
# Đánh giá mô hình cảm xúc
emotion_metrics = trainer_emotion.evaluate()
print("Emotion Classification Metrics:", emotion_metrics)

# Đánh giá mô hình chủ đề
topic_metrics = trainer_topic.evaluate()
print("Topic Classification Metrics:", topic_metrics)


In [ ]:
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model_emotion(**inputs)
    predicted = torch.argmax(outputs.logits, dim=1)
    return predicted.item()

def predict_topic(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model_topic(**inputs)
    predicted = torch.argmax(outputs.logits, dim=1)
    return predicted.item()

# Ví dụ dự đoán
sample_text = "Giảng viên giảng bài rất dễ hiểu và nhiệt tình."
print("Emotion:", predict_emotion(sample_text))
print("Topic:", predict_topic(sample_text))
